In [4]:
import sys
sys.path+=['/Users/lunzhang/anaconda3/lib/python3.6/site-packages']
import DSGRN
import os
import json

In [5]:
nwStr = "Mdm2:p53 \n ATM:~Wip1 \n Wip1:p53 \n Chk2:(ATM)(~Wip1) \n p53:(ATM+Chk2)(~Mdm2)"
network = DSGRN.Network(nwStr)
DSGRN.DrawGraph(network)
parameterGraph = DSGRN.ParameterGraph(network)
parameterGraph.size()

803520

In [6]:
oscillatory = []
size = parameterGraph.size()
for i in range(size):
    parameterSample = parameterGraph.parameter(i)
    domainGraph = DSGRN.DomainGraph(parameterSample)
    morseDecomposition = DSGRN.MorseDecomposition(domainGraph.digraph())
    morseGraph = DSGRN.MorseGraph(domainGraph, morseDecomposition)
    l = morseGraph.poset().size()
    if i%100000==0:
        print(i)
    for j in range(l):
        if morseGraph.annotation(j)[0].startswith('FC') and len(morseGraph.poset().children(j))==0:
            oscillatory.append(parameterSample)
            break

0
100000
200000
300000
400000
500000
600000
700000
800000


In [7]:
# suppose there are at most 2 thresholds
def stringifyParser(l):
    ret =  []
    l = eval(l)
    #print(l)
    for i in range(len(l)):
        inputNo = l[i][1][0]  
        m=outputNo = l[i][1][1] 
        polyNo = pow(2,inputNo)
        totalLen = polyNo*outputNo

        logic = l[i][1][2]
        logic = bin(int(logic,16))[2:].zfill(totalLen)

        lower = []
        middle = []
        upper = []
        thetas = l[i][2]
        temp = []
        for j in range(0,len(logic),m):
            polyIndex = polyNo - int(j/m) - 1
            piece = logic[j:j+m]
            #print(piece)
            if len(set(piece)) == 2:
                middle.append(polyIndex)
            elif set(piece) == set('1'):
                upper.append(polyIndex)
            else:
                lower.append(polyIndex)

        if len(thetas)==1:
            temp = sorted(lower) + [-thetas[0]-1] + sorted(upper)
        else:
            temp = sorted(lower) + [-thetas[0]-1] + sorted(middle) + [-thetas[1]-1]+sorted(upper)
            #print(temp)

        ret.append(tuple(temp))
    return ret

In [14]:
pnlist = [stringifyParser(x.stringify()) for x in oscillatory]

In [15]:
with open('osciP53pnlist.json','w') as file:
    json.dump(pnlist, file)